In [12]:
#!pip install langchain
#!pip install langchain_community
#!pip install langchain_openai


# Imports

In [1]:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings


# Warning control
import warnings
warnings.filterwarnings("ignore")

# Properties

In [2]:
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "nimda123"
NEO4J_DATABASE = "neo4j"

OPENAI_API_KEY=""


In [3]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [4]:
kg.query("""
  SHOW VECTOR INDEXES
  """
)

[{'id': 3,
  'name': 'attack_index',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Attack'],
  'properties': ['descriptionEmbedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 7, 21, 21, 34, 2, 179000000, tzinfo=<UTC>),
  'readCount': 14},
 {'id': 4,
  'name': 'malware_index',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Malware'],
  'properties': ['descriptionEmbedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 8, 16, 19, 57, 33, 98000000, tzinfo=<UTC>),
  'readCount': 2391}]

- create a vector index for attack,
- use openai to store the vector indexes

In [5]:
kg.query("""
         CREATE VECTOR INDEX `attack_index` IF NOT EXISTS
          FOR (a:Attack) ON (a.descriptionEmbedding) 
          OPTIONS { indexConfig: {
            `vector.dimensions`: 1536,
            `vector.similarity_function`: 'cosine'    
         }}
""")

[]

In [6]:
kg.query("""
         CREATE VECTOR INDEX `malware_index` IF NOT EXISTS
          FOR (a:Malware) ON (a.descriptionEmbedding) 
          OPTIONS { indexConfig: {
            `vector.dimensions`: 1536,
            `vector.similarity_function`: 'cosine'    
         }}
""")

[]

In [7]:
kg.query("""
  SHOW VECTOR INDEXES
  """
)

[{'id': 3,
  'name': 'attack_index',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Attack'],
  'properties': ['descriptionEmbedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 7, 21, 21, 34, 2, 179000000, tzinfo=<UTC>),
  'readCount': 14},
 {'id': 4,
  'name': 'malware_index',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Malware'],
  'properties': ['descriptionEmbedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 8, 16, 19, 57, 33, 98000000, tzinfo=<UTC>),
  'readCount': 2391}]

In [8]:
#kg.query("""
#  DROP INDEX attack_index
#  """
#)

# Generate the Embeddings

In [9]:

import csv
from openai import OpenAI

client = OpenAI(api_key = OPENAI_API_KEY)



"""
node: Type of node
prop: property of the text
prop_emb: property to store embeddings
kg: Neo4jGraph onject
"""

def get_node_textual_data(kg, node,prop = 'description',prop_emb='descriptionEmbedding'):
    
    cypher = f"""
        MATCH (a:{node}) WHERE a.{prop_emb} IS NULL
        RETURN a.id AS id, a.{prop} AS {prop}
    """
    params = {
        'node' : node,
        'prop' : prop,
        'prop_emb' : prop_emb
    }
    
    nodes_to_embed = kg.query(cypher, params)
    return nodes_to_embed

"""
text_data: Text data to embed
model : embedding model
"""
def get_embeddings(text_data, emb_model="text-embedding-ada-002", client =client):
    
    response = client.embeddings.create(input =text_data, model=emb_model)
    
    return response.data[0].embedding


"""
csv_path: location to store the csv file
prop: property of the text,
nodes_to_embed: list of dictionary of extracted node information
"""
def extract_and_save_embeddings(csv_path, nodes_to_embed, prop = 'description'):
    with open(csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['id', 'embedding'])
        
        for node in nodes_to_embed:
            textual_data = node.get(prop)
            try:
                embeddings = get_embeddings(textual_data, "text-embedding-ada-002")
                writer.writerow([node['id'], embeddings])
                print("generate embeddings for id - " + node['id'])
            except:
                continue

        return csv_path
      
"""
node: Type of node

prop_emb: property to store embeddings
kg: Neo4jGraph object
csv_path: location to store the 
"""
def store_embeddings_from_csv(kg, csv_path, node, prop_emb):
    with open(csv_path, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row
        for row in reader:
            id = row[0]
            embedding = [float(x) for x in row[1][1:-1].split(',')]
            cypher = f"""
                MATCH (a:{node} {{id: $id}})
                SET a.{prop_emb} = $embedding
            """
            params = {'id': id, 'embedding': embedding}
            kg.query(cypher, params)
            print(f'Added embeddings to node - {id}')
    return "Success"

In [10]:
"""# create embeddings for Attack 
# get nodes to embed
nodes_to_embed = get_node_textual_data(kg, 'Attack', 'description', 'descriptionEmbedding')

# save embeddings
extract_and_save_embeddings('Attack_embeddings.csv', nodes_to_embed, 'description')"""

"# create embeddings for Attack \n# get nodes to embed\nnodes_to_embed = get_node_textual_data(kg, 'Attack', 'description', 'descriptionEmbedding')\n\n# save embeddings\nextract_and_save_embeddings('Attack_embeddings.csv', nodes_to_embed, 'description')"

In [11]:
"""# create embeddings for Malware 
# get nodes to embed
nodes_to_embed = get_node_textual_data(kg, 'Malware', 'description', 'descriptionEmbedding')

# save embeddings
extract_and_save_embeddings('Malware_embeddings.csv', nodes_to_embed, 'description')"""

"# create embeddings for Malware \n# get nodes to embed\nnodes_to_embed = get_node_textual_data(kg, 'Malware', 'description', 'descriptionEmbedding')\n\n# save embeddings\nextract_and_save_embeddings('Malware_embeddings.csv', nodes_to_embed, 'description')"

In [12]:
"""# load embeddings to Attack

store_embeddings_from_csv(kg, 'Attack_embeddings.csv', 'Attack', 'descriptionEmbedding')"""

Added embeddings to node - attack-pattern--0458aab9-ad42-4eac-9e22-706a95bafee2
Added embeddings to node - attack-pattern--045d0922-2310-4e60-b5e4-3302302cb3c5
Added embeddings to node - attack-pattern--0470e792-32f8-46b0-a351-652bc35e9336
Added embeddings to node - attack-pattern--04a5a8ab-3bc8-4c83-95c9-55274a89786d
Added embeddings to node - attack-pattern--04ee0cb7-dac3-4c6c-9387-4c6aa096f4cf
Added embeddings to node - attack-pattern--04ef4356-8926-45e2-9441-634b6f3dcecb
Added embeddings to node - attack-pattern--04fd5427-79c7-44ea-ae13-11b24778ff1c
Added embeddings to node - attack-pattern--0533ab23-3f7d-463f-9bd8-634d27e4dee1
Added embeddings to node - attack-pattern--06780952-177c-4247-b978-79c357fb311f
Added embeddings to node - attack-pattern--06c00069-771a-4d57-8ef5-d3718c1a8771
Added embeddings to node - attack-pattern--0708ae90-d0eb-4938-9a76-d0fc94f6eec1
Added embeddings to node - attack-pattern--086952c4-5b90-4185-b573-02bad8e11953
Added embeddings to node - attack-patter

Added embeddings to node - attack-pattern--2169ba87-1146-4fc7-a118-12b72251db7e
Added embeddings to node - attack-pattern--21875073-b0ee-49e3-9077-1e2a885359af
Added embeddings to node - attack-pattern--22522668-ddf6-470b-a027-9d6866679f67
Added embeddings to node - attack-pattern--22905430-4901-4c2a-84f6-98243cb173f8
Added embeddings to node - attack-pattern--232a7e42-cd6e-4902-8fe9-2960f529dd4d
Added embeddings to node - attack-pattern--232b7f21-adf9-4b42-b936-b9d6f7df856e
Added embeddings to node - attack-pattern--2339cf19-8f1e-48f7-8a91-0262ba547b6f
Added embeddings to node - attack-pattern--241814ae-de3f-4656-b49e-f9a80764d4b7
Added embeddings to node - attack-pattern--24286c33-d4a4-4419-85c2-1d094a896c26
Added embeddings to node - attack-pattern--246fd3c7-f5e3-466d-8787-4c13d9e3b61c
Added embeddings to node - attack-pattern--24769ab5-14bd-4f4e-a752-cfb185da53ee
Added embeddings to node - attack-pattern--24bfaeba-cb0d-4525-b3dc-507c77ecec41
Added embeddings to node - attack-patter

Added embeddings to node - attack-pattern--46944654-fcc1-4f63-9dad-628102376586
Added embeddings to node - attack-pattern--478aa214-2ca7-4ec0-9978-18798e514790
Added embeddings to node - attack-pattern--47f2d673-ca62-47e9-929b-1b0be9657611
Added embeddings to node - attack-pattern--4933e63b-9b77-476e-ab29-761bc5b7d15a
Added embeddings to node - attack-pattern--494ab9f0-36e0-4b06-b10d-57285b040a06
Added embeddings to node - attack-pattern--4a2975db-414e-4c0c-bd92-775987514b4b
Added embeddings to node - attack-pattern--4a5b7ade-8bb5-4853-84ed-23f262002665
Added embeddings to node - attack-pattern--4ab929c6-ee2d-4fb5-aab4-b14be2ed7179
Added embeddings to node - attack-pattern--4ae4f953-fe58-4cc8-a327-33257e30a830
Added embeddings to node - attack-pattern--4b74a1d4-b0e9-4ef1-93f1-14ecc6e2f5b5
Added embeddings to node - attack-pattern--4bc31b94-045b-4752-8920-aebaebdb6470
Added embeddings to node - attack-pattern--4be89c7c-ace6-4876-9377-c8d54cef3d63
Added embeddings to node - attack-patter

Added embeddings to node - attack-pattern--6c2957f9-502a-478c-b1dd-d626c0659413
Added embeddings to node - attack-pattern--6d4a7fb3-5a24-42be-ae61-6728a2b581f6
Added embeddings to node - attack-pattern--6e3bd510-6b33-41a4-af80-2d80f3ee0071
Added embeddings to node - attack-pattern--6e561441-8431-4773-a9b8-ccf28ef6a968
Added embeddings to node - attack-pattern--6e6845c2-347a-4a6f-a2d1-b74a18ebd352
Added embeddings to node - attack-pattern--6ee2dc99-91ad-4534-a7d8-a649358c331f
Added embeddings to node - attack-pattern--6fa224c7-5091-4595-bf15-3fc9fe2f2c7c
Added embeddings to node - attack-pattern--6faf650d-bf31-4eb4-802d-1000cf38efaf
Added embeddings to node - attack-pattern--6fb6408c-0db3-41d9-a3a1-a32e5f16454e
Added embeddings to node - attack-pattern--6ff403bc-93e3-48be-8687-e102fdba8c88
Added embeddings to node - attack-pattern--7007935a-a8a7-4c0b-bd98-4e85be8ed197
Added embeddings to node - attack-pattern--707399d6-ab3e-4963-9315-d9d3818cd6a0
Added embeddings to node - attack-patter

Added embeddings to node - attack-pattern--9664ad0e-789e-40ac-82e2-d7b17fbe8fb3
Added embeddings to node - attack-pattern--970a3432-3237-47ad-bcca-7d8cbb217736
Added embeddings to node - attack-pattern--970cdb5c-02fb-4c38-b17e-d6327cf3c810
Added embeddings to node - attack-pattern--98034fef-d9fb-4667-8dc4-2eab6231724c
Added embeddings to node - attack-pattern--98be40f2-c86b-4ade-b6fc-4964932040e5
Added embeddings to node - attack-pattern--99709758-2b96-48f2-a68a-ad7fbd828091
Added embeddings to node - attack-pattern--9a60a291-8960-4387-8a4a-2ab5c18bb50b
Added embeddings to node - attack-pattern--9b52fca7-1a36-4da0-b62d-da5bd83b4d69
Added embeddings to node - attack-pattern--9b99b83a-1aac-4e29-b975-b374950551a3
Added embeddings to node - attack-pattern--9c306d8d-cde7-4b4c-b6e8-d0bb16caca36
Added embeddings to node - attack-pattern--9c45eaa3-8604-4780-8988-b5074dbb9ecd
Added embeddings to node - attack-pattern--9c99724c-a483-4d60-ad9d-7f004e42e8e8
Added embeddings to node - attack-patter

Added embeddings to node - attack-pattern--be2dcee9-a7a7-4e38-afd6-21b31ecc3d63
Added embeddings to node - attack-pattern--bed04f7d-e48a-4e76-bd0f-4c57fe31fc46
Added embeddings to node - attack-pattern--bef8aaee-961d-4359-a308-4c2182bcedff
Added embeddings to node - attack-pattern--bf147104-abf9-4221-95d1-e81585859441
Added embeddings to node - attack-pattern--bf176076-b789-408e-8cba-7275e81c0ada
Added embeddings to node - attack-pattern--bf1b6176-597c-4600-bfcd-ac989670f96b
Added embeddings to node - attack-pattern--bf90d72c-c00b-45e3-b3aa-68560560d4c5
Added embeddings to node - attack-pattern--bf96a5a3-3bce-43b7-8597-88545984c07b
Added embeddings to node - attack-pattern--c071d8c1-3b3a-4f22-9407-ca4e96921069
Added embeddings to node - attack-pattern--c0a384a4-9a25-40e1-97b6-458388474bc8
Added embeddings to node - attack-pattern--c0df6533-30ee-4a4a-9c6d-17af5abdf0b2
Added embeddings to node - attack-pattern--c0dfe7b0-b873-4618-9ff8-53e31f70907f
Added embeddings to node - attack-patter

Added embeddings to node - attack-pattern--e5cc9e7a-e61a-46a1-b869-55fb6eab058e
Added embeddings to node - attack-pattern--e5d550f3-2202-4634-85f2-4a200a1d49b3
Added embeddings to node - attack-pattern--e624264c-033a-424d-9fd7-fc9c3bbdb03e
Added embeddings to node - attack-pattern--e6415f09-df0e-48de-9aba-928c902b7549
Added embeddings to node - attack-pattern--e64c62cf-9cd7-4a14-94ec-cdaac43ab44b
Added embeddings to node - attack-pattern--e6919abc-99f9-4c6c-95a5-14761e7b2add
Added embeddings to node - attack-pattern--e6f19759-dde3-47fc-99cc-d9f5fa4ade60
Added embeddings to node - attack-pattern--e74de37c-a829-446c-937d-56a44f0e9306
Added embeddings to node - attack-pattern--e7cbc1de-1f79-48ee-abfd-da1241c65a15
Added embeddings to node - attack-pattern--e7eab98d-ae11-4491-bd28-a53ba875865a
Added embeddings to node - attack-pattern--e848506b-8484-4410-8017-3d235a52f5b3
Added embeddings to node - attack-pattern--e8a0a025-3601-4755-abfb-8d08283329fb
Added embeddings to node - attack-patter

'Success'

In [13]:
"""# load embeddings to Malware

store_embeddings_from_csv(kg, 'Malware_embeddings.csv', 'Malware', 'descriptionEmbedding')"""

Added embeddings to node - malware--007b44b6-e4c5-480b-b5b9-56f2081b1b7b
Added embeddings to node - malware--00806466-754d-44ea-ad6f-0caf59cb8556
Added embeddings to node - malware--00c3bfcb-99bd-4767-8c03-b08f585f5c8a
Added embeddings to node - malware--00e7d565-9883-4ee5-b642-8fd17fd6a3f5
Added embeddings to node - malware--01dbc71d-0ee8-420d-abb4-3dfb6a4bf725
Added embeddings to node - malware--023254de-caaf-4a05-b2c7-e4e2f283f7a5
Added embeddings to node - malware--039814a0-88de-46c5-a4fb-b293db21880a
Added embeddings to node - malware--03acae53-9b98-46f6-b204-16b930839055
Added embeddings to node - malware--03ea629c-517a-41e3-94f8-c7e5368cf8f4
Added embeddings to node - malware--03eb4a05-6a02-43f6-afb7-3c7835501828
Added embeddings to node - malware--04227b24-7817-4de1-9050-b7b1b57f5866
Added embeddings to node - malware--04378e79-4387-468a-a8f7-f974b8254e44
Added embeddings to node - malware--049ff071-0b3c-4712-95d2-d21c6aa54501
Added embeddings to node - malware--04cecafd-cb5f-4

Added embeddings to node - malware--40a1b8ec-7295-416c-a6b1-68181d86f120
Added embeddings to node - malware--40d3e230-ed32-469f-ba89-be70cc08ab39
Added embeddings to node - malware--414dc555-c79e-4b24-a2da-9b607f7eaf16
Added embeddings to node - malware--4189a679-72ed-4a89-a57c-7f689712ecf8
Added embeddings to node - malware--425771c5-48b4-4ecd-9f95-74ed3fc9da59
Added embeddings to node - malware--43155329-3edf-47a6-9a14-7dac899b01e4
Added embeddings to node - malware--43213480-78f7-4fb3-976f-d48f5f6a4c2a
Added embeddings to node - malware--432555de-63bf-4f2a-a3fa-f720a4561078
Added embeddings to node - malware--4327aff5-f194-440c-b499-4d9730cc1eab
Added embeddings to node - malware--434ba392-ebdc-488b-b1ef-518deea65774
Added embeddings to node - malware--44c75271-0e4d-496f-ae0a-a6d883a42a65
Added embeddings to node - malware--44e2a842-415b-47f4-8549-83fbdb8a5674
Added embeddings to node - malware--452da2d9-706c-4185-ad6f-f5edaf4b9f48
Added embeddings to node - malware--454fe82d-6fd2-4

Added embeddings to node - malware--687c23e4-4e25-4ee7-a870-c5e002511f54
Added embeddings to node - malware--68dca94f-c11d-421e-9287-7c501108e18c
Added embeddings to node - malware--691c60e2-273d-4d56-9ce6-b67e0f8719ad
Added embeddings to node - malware--69d6f4a9-fcf0-4f51-bca7-597c51ad0bb8
Added embeddings to node - malware--6a0d0ea9-b2c4-43fe-a552-ac41a3009dc5
Added embeddings to node - malware--6a0ef5d4-fc7c-4dda-85d7-592e4dbdc5d9
Added embeddings to node - malware--6a21e3a4-5ffe-4581-af9a-6a54c7536f44
Added embeddings to node - malware--6a92d80f-cc65-45f6-aa66-3cdea6786b3c
Added embeddings to node - malware--6b616fc1-1505-48e3-8b2c-0d19337bff38
Added embeddings to node - malware--6b62e336-176f-417b-856a-8552dd8c44e1
Added embeddings to node - malware--6ba1d7ae-d60b-43e6-9f08-a8b787e9d9cb
Added embeddings to node - malware--6c2550d5-a01a-4bbb-a004-6ead348ba623
Added embeddings to node - malware--6c575670-d14c-4c7f-9b9d-fd1b363e255d
Added embeddings to node - malware--6cd07296-14aa-4

Added embeddings to node - malware--a545456a-f9a7-47ad-9ea6-8b017def38d1
Added embeddings to node - malware--a5528622-3a8a-4633-86ce-8cdaf8423858
Added embeddings to node - malware--a5575606-9b85-4e3d-9cd2-40ef30e3672d
Added embeddings to node - malware--a5818d36-e9b0-46da-842d-b727a5e36ea6
Added embeddings to node - malware--a5e91d50-24fa-44ec-9894-39a88f658cea
Added embeddings to node - malware--a60657fa-e2e7-4f8f-8128-a882534ae8c5
Added embeddings to node - malware--a705b085-1eae-455e-8f4d-842483d814eb
Added embeddings to node - malware--a7881f21-e978-4fe4-af56-92c9416a2616
Added embeddings to node - malware--a8839c95-029f-44cf-8f3d-a3cf2039e927
Added embeddings to node - malware--a8a778f5-0035-4870-bb25-53dc05029586
Added embeddings to node - malware--a8d3d497-2da9-4797-8e0b-ed176be08654
Added embeddings to node - malware--aa1462a1-d065-416c-b354-bedd04998c7f
Added embeddings to node - malware--aad11e34-02ca-4220-91cd-2ed420af4db3
Added embeddings to node - malware--aae22730-e571-4

Added embeddings to node - malware--e48df773-7c95-4a4c-ba70-ea3d15900148
Added embeddings to node - malware--e494ad79-37ee-4cd0-866b-299c521d8b94
Added embeddings to node - malware--e4feffc2-53d1-45c9-904e-adb9faca0d15
Added embeddings to node - malware--e669bb87-f773-4c7b-bfcc-a9ffebfdd8d4
Added embeddings to node - malware--e6ef745b-077f-42e1-a37d-29eecff9c754
Added embeddings to node - malware--e7863f5d-cb6a-4f81-8804-0a635eec160a
Added embeddings to node - malware--e7a5229f-05eb-440e-b982-9a6d2b2b87c8
Added embeddings to node - malware--e811ff6a-4cef-4856-a6ae-a7daf9ed39ae
Added embeddings to node - malware--e8268361-a599-4e45-bd3f-71c8c7e700c0
Added embeddings to node - malware--e8545794-b98c-492b-a5b3-4b5a02682e37
Added embeddings to node - malware--e85cae1a-bce3-4ac4-b36b-b00acac0567b
Added embeddings to node - malware--e928333f-f3df-4039-9b8b-556c2add0e42
Added embeddings to node - malware--e9595678-d269-469e-ae6b-75e49259de63
Added embeddings to node - malware--e9e9bfe2-76f4-4

'Success'

In [14]:
question = "The attack should be an attack that happend on power grid or smart grid, if it is not an attack that happend on power grid or smart grid please do not recommend  \
and check if this attack is possilbe on the Wind power plant, select attack that were only related to this"

In [15]:
question = "Attack on Generating Unit"

In [16]:


question_embeddings = get_embeddings(question)

a = kg.query(f"""
    CALL db.index.vector.queryNodes(
        'malware_index', 
        $top_k, 
        {question_embeddings}
        ) YIELD node AS malware, score
    RETURN malware.id as id , malware.name as name, score, malware.description as description
    """, 
    params={
            "question_embeddings": question_embeddings,
            "top_k": 1000
            })
a

[{'id': 'malware--e401d4fe-f0c9-44f0-98e6-f93487678808',
  'name': 'Industroyer',
  'score': 0.893214225769043,
  'description': '[Industroyer](https://attack.mitre.org/software/S0604) is a sophisticated malware framework designed to cause an impact to the working processes of Industrial Control Systems (ICS), specifically components used in electrical substations.(Citation: ESET Industroyer) [Industroyer](https://attack.mitre.org/software/S0604) was used in the attacks on the Ukrainian power grid in December 2016.(Citation: Dragos Crashoverride 2017) This is the first publicly known malware specifically designed to target and impact operations in the electric grid.(Citation: Dragos Crashoverride 2018)'},
 {'id': 'malware--9af05de0-bc09-4511-a350-5eb8b06185c1',
  'name': 'BadPatch',
  'score': 0.889999270439148,
  'description': '[BadPatch](https://attack.mitre.org/software/S0337) is a Windows Trojan that was used in a Gaza Hackers-linked campaign.(Citation: Unit 42 BadPatch Oct 2017)'

# Create connections 

In [17]:


cypher = """

    MATCH (n:Interface)
    RETURN n.id as id, n.name AS name, n.comment AS comment, n.description as description

"""

powergrid_nodes = kg.query(cypher)

In [18]:
powergrid_nodes

[{'id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:EquipmentContainer;1',
  'name': None,
  'comment': None,
  'description': None},
 {'id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:ConnectivityNodeContainer;1',
  'name': 'ConnectivityNodeContainer',
  'comment': 'Adapted from CIM and https://github.com/smart-data-models/dataModel.EnergyCIM/ data models',
  'description': 'A base class for all objects that may contain connectivity nodes or topological nodes.'},
 {'id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:Equipment;1',
  'name': 'relatedEquipment',
  'comment': 'connection to an equipment.',
  'description': None},
 {'id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:BaseVoltage;1',
  'name': 'BaseVoltage',
  'comment': 'Base voltage of the transformer end.  This is essential for PU calculation.',
  'description': None},
 {'id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:IdentifiedObject;1',
  'name': 'IdentifiedObject',
  'comment': 'Adapted from CIM and https://github.com/smart-data-models/dataModel.En

In [19]:
# Now i do similarity analysis on Interface and the Malware index

In [20]:
def get_similarity_from_question(question):
    
    
    question_embeddings = get_embeddings(question)
    
    results = kg.query(f"""
    CALL db.index.vector.queryNodes(
        'malware_index', 
        $top_k, 
        {question_embeddings}
        ) YIELD node AS malware, score
    RETURN malware.id as id, malware.name as name, score, malware.description as description
    """, 
    params={
            "question_embeddings": question_embeddings,
            "top_k": 100
            })
    return results


def create_relation_from_question(kg, source_id, target_id, relation="possible_attack"):
    status = None
    if source_id is not None and target_id is not None:
        cypher = f"""
        MATCH (a:Malware {{id: $source_id}}), (b:Interface {{id: $target_id}})
        MERGE (a)-[r:{relation}]->(b)
        RETURN type(r)
        """
        params = {
            "source_id": source_id,
            "target_id": target_id
        }
        
        status = kg.query(cypher, params)
        #print(f"Relation created between source_id - {source_id} and target_id - {target_id} with relation - {relation}")
    return status


# Links creation

In [52]:

for node in powergrid_nodes:
    count = 0 
    question = ""
    if (None != node.get('name')):
        question += "Attack on " + node.get('name') 
    """if (None != node.get('comment') and not node.get('comment').startswith("Adapted from CIM")):
        question += " - " + node.get('comment')
        
    if (None != node.get('description') and not node.get('description').startswith("Adapted from CIM")):
        question += " - " + node.get('description')"""

    if(question):
        similarity_results = get_similarity_from_question(question)
        for result in similarity_results:
            if(result.get('score') >0.87):
                create_relation_from_question(kg,result.get("id"), node.get("id"), "possible_attack")
                count +=1
            else:
                break
    print ("Total Number of Relations =" + str(count))
                
            
   

Total Number of Relations =0
Total Number of Relations =5
Total Number of Relations =23
Total Number of Relations =100
Total Number of Relations =22
Total Number of Relations =100
Total Number of Relations =100
Total Number of Relations =1
Total Number of Relations =100
Total Number of Relations =100
Total Number of Relations =53
Total Number of Relations =100
Total Number of Relations =100
Total Number of Relations =30
Total Number of Relations =100
Total Number of Relations =1
Total Number of Relations =100
Total Number of Relations =0
Total Number of Relations =0
Total Number of Relations =29
Total Number of Relations =100
Total Number of Relations =26
Total Number of Relations =100
Total Number of Relations =52
Total Number of Relations =100
Total Number of Relations =100
Total Number of Relations =100
Total Number of Relations =100
Total Number of Relations =100
Total Number of Relations =100
Total Number of Relations =100
Total Number of Relations =100
Total Number of Relations =

In [22]:
result.get('score')

0.8699622750282288

In [23]:
type(result.get('score'))

float

In [24]:
# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI


In [25]:
VECTOR_INDEX_NAME = 'malware_index'
VECTOR_NODE_LABEL = 'Malware'
VECTOR_SOURCE_PROPERTY = 'description'
VECTOR_EMBEDDING_PROPERTY = 'descriptionEmbedding'


In [26]:
malware_vector_store = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)

In [27]:
retriever = malware_vector_store.as_retriever()


plain_chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY), 
    chain_type="stuff", 
    retriever=retriever
)

In [28]:
question = "what are the malware that can attack Generating Unit?"

In [29]:
import textwrap

In [30]:
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties: Attack {type: STRING, name: STRING,
descriptionEmbedding: LIST, description: STRING,
external_references: STRING, id: STRING, x_mitre_detection:
STRING} Campaign {type: STRING, name: STRING, description:
STRING, external_references: STRING, id: STRING} Action
{type: STRING, name: STRING, description: STRING,
external_references: STRING, id: STRING} Intrusion {name:
STRING, type: STRING, description: STRING,
external_references: STRING, id: STRING} Malware {type:
STRING, id: STRING, name: STRING, descriptionEmbedding:
LIST, x_mitre_platforms: STRING, description: STRING,
external_references: STRING} Tool {id: STRING, name: STRING,
type: STRING, x_mitre_platforms: STRING, description:
STRING, external_references: STRING} DataComponent {type:
STRING, name: STRING, description: STRING, id: STRING}
Interface {id: STRING, @type: STRING, name: STRING, comment:
STRING, description: STRING, type: STRING} Property {type:
STRING, id: STRING, name: STRING, comment: STRING, schema:

In [31]:

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to 
query a graph database. 
Instructions:
Use only the provided relationship types and properties in the 
schema. Do not use any other relationship types or properties that 
are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than 
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher 
statements for particular questions:

# What Malwares that can attack a GeneratingUnit?
MATCH (maware:Malware)-[:possible_attack]->(node:Interface)
    WHERE node.name = 'GeneratingUnit'
RETURN maware.name, maware.description



The question is:
{question}"""

In [32]:
from dotenv import load_dotenv
import os

import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [33]:
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

In [34]:
cypherChain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY),
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    top_k = 50,
)

In [35]:
def prettyCypherChain(question: str) -> str:
    response = cypherChain.run(question)
    print(textwrap.fill(response, 60))
    return response

In [36]:
prettyCypherChain("What Malwares that can attack a HydroPowerPlant (A hydro power station which can generate or pump. When generating, the generator turbines receive water from an upper reservoir. When pumping, the pumps receive their water from a lower reservoir)?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (malware:Malware)-[:possible_attack]->(interface:Interface)
WHERE interface.name = 'HydroPowerPlant'
RETURN malware.name, malware.description
Full Context:
[{'malware.name': 'Industroyer', 'malware.description': '[Industroyer](https://attack.mitre.org/software/S0604) is a sophisticated malware framework designed to cause an impact to the working processes of Industrial Control Systems (ICS), specifically components used in electrical substations.(Citation: ESET Industroyer) [Industroyer](https://attack.mitre.org/software/S0604) was used in the attacks on the Ukrainian power grid in December 2016.(Citation: Dragos Crashoverride 2017) This is the first publicly known malware specifically designed to target and impact operations in the electric grid.(Citation: Dragos Crashoverride 2018)'}, {'malware.name': 'Carbanak', 'malware.description': '[Carbanak](https://attack.mitre.org/software/S0030) is a full-featured, remote b


> Finished chain.
Industroyer and Stuxnet are malwares that can attack a
HydroPowerPlant.


'Industroyer and Stuxnet are malwares that can attack a HydroPowerPlant.'

In [37]:
prettyCypherChain("What Malwares that can attack WindPowerPlant (Wind power plant)?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (malware:Malware)-[:possible_attack]->(node:Interface)
WHERE node.name = 'WindPowerPlant'
RETURN malware.name, malware.description
Full Context:
[{'malware.name': 'MoonWind', 'malware.description': '[MoonWind](https://attack.mitre.org/software/S0149) is a remote access tool (RAT) that was used in 2016 to target organizations in Thailand. (Citation: Palo Alto MoonWind March 2017)'}, {'malware.name': 'Machete', 'malware.description': '[Machete](https://attack.mitre.org/software/S0409) is a cyber espionage toolset used by [Machete](https://attack.mitre.org/groups/G0095). It is a Python-based backdoor targeting Windows machines that was first observed in 2010.(Citation: ESET Machete July 2019)(Citation: Securelist Machete Aug 2014)(Citation: 360 Machete Sep 2020)'}, {'malware.name': 'Helminth', 'malware.description': '[Helminth](https://attack.mitre.org/software/S0170) is a backdoor that has at least two variants - one wr


> Finished chain.
I don't know the answer.


"I don't know the answer."

In [38]:


prettyCypherChain("What Malwares that can attack WindPowerPlant?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (malware:Malware)-[:possible_attack]->(node:Interface)
    WHERE node.name = 'WindPowerPlant'
RETURN malware.name, malware.description
Full Context:
[{'malware.name': 'MoonWind', 'malware.description': '[MoonWind](https://attack.mitre.org/software/S0149) is a remote access tool (RAT) that was used in 2016 to target organizations in Thailand. (Citation: Palo Alto MoonWind March 2017)'}, {'malware.name': 'Machete', 'malware.description': '[Machete](https://attack.mitre.org/software/S0409) is a cyber espionage toolset used by [Machete](https://attack.mitre.org/groups/G0095). It is a Python-based backdoor targeting Windows machines that was first observed in 2010.(Citation: ESET Machete July 2019)(Citation: Securelist Machete Aug 2014)(Citation: 360 Machete Sep 2020)'}, {'malware.name': 'Helminth', 'malware.description': '[Helminth](https://attack.mitre.org/software/S0170) is a backdoor that has at least two variants - on


> Finished chain.
I don't know the answer.


"I don't know the answer."

In [39]:


prettyCypherChain("What Malwares that can attack GeneratingUnit?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (malware:Malware)-[:possible_attack]->(interface:Interface)
WHERE interface.name = 'GeneratingUnit'
RETURN malware.name, malware.description
Full Context:
[{'malware.name': 'BadPatch', 'malware.description': '[BadPatch](https://attack.mitre.org/software/S0337) is a Windows Trojan that was used in a Gaza Hackers-linked campaign.(Citation: Unit 42 BadPatch Oct 2017)'}, {'malware.name': 'Hydraq', 'malware.description': '[Hydraq](https://attack.mitre.org/software/S0203) is a data-theft trojan first used by [Elderwood](https://attack.mitre.org/groups/G0066) in the 2009 Google intrusion known as Operation Aurora, though variations of this trojan have been used in more recent campaigns by other Chinese actors, possibly including [APT17](https://attack.mitre.org/groups/G0025).(Citation: MicroFocus 9002 Aug 2016)(Citation: Symantec Elderwood Sept 2012)(Citation: Symantec Trojan.Hydraq Jan 2010)(Citation: ASERT Seven Pointed Da

'BadPatch, Hydraq, GeminiDuke, Rising Sun, POWERSTATS, DDKONG, Industroyer2, PLAINTEE, RogueRobin, Final1stspy, BlackEnergy, PowerPunch, YAHOYAH, Pteranodon, Industroyer, QUADAGENT can attack GeneratingUnit.'

In [40]:

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to 
query a graph database. 
Instructions:
Use only the provided relationship types and properties in the 
schema. Do not use any other relationship types or properties that 
are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than 
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
In the final output give only names
Examples: Here are a few examples of generated Cypher 
statements for particular questions:

# What Malwares that can attack a GeneratingUnit?
MATCH (maware:Malware)-[:possible_attack]->(node:Interface)
    WHERE node.name = 'GeneratingUnit'
RETURN maware.name, maware.description

#How to defend against Indostroyer malware?
MATCH (m:Malware)-[:uses]->(a:Attack)<-[:mitigates]-(act:Action)
    WHERE m.name = 'Indostroyer'
RETURN act.description


The question is:
{question}"""

In [41]:

prettyCypherChain("Give Malwares that can attack GeneratingUnit ?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (malware:Malware)-[:possible_attack]->(interface:Interface)
WHERE interface.name = 'GeneratingUnit'
RETURN malware.name, malware.description
Full Context:
[{'malware.name': 'BadPatch', 'malware.description': '[BadPatch](https://attack.mitre.org/software/S0337) is a Windows Trojan that was used in a Gaza Hackers-linked campaign.(Citation: Unit 42 BadPatch Oct 2017)'}, {'malware.name': 'Hydraq', 'malware.description': '[Hydraq](https://attack.mitre.org/software/S0203) is a data-theft trojan first used by [Elderwood](https://attack.mitre.org/groups/G0066) in the 2009 Google intrusion known as Operation Aurora, though variations of this trojan have been used in more recent campaigns by other Chinese actors, possibly including [APT17](https://attack.mitre.org/groups/G0025).(Citation: MicroFocus 9002 Aug 2016)(Citation: Symantec Elderwood Sept 2012)(Citation: Symantec Trojan.Hydraq Jan 2010)(Citation: ASERT Seven Pointed Da

'BadPatch, Hydraq, GeminiDuke, Rising Sun, POWERSTATS, DDKONG, Industroyer2, PLAINTEE, RogueRobin, Final1stspy, BlackEnergy, PowerPunch, YAHOYAH, Pteranodon, Industroyer, QUADAGENT.'

In [42]:
prettyCypherChain("Give Malwares that can attack DCConverterUnit (The DC converter unit belonging of the substation.)?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (malware:Malware)-[:possible_attack]->(node:Interface)
WHERE node.name = 'DCConverterUnit'
RETURN malware.name, malware.description
Full Context:
[{'malware.name': 'Industroyer2', 'malware.description': '[Industroyer2](https://attack.mitre.org/software/S1072) is a compiled and static piece of malware that has the ability to communicate over the IEC-104 protocol. It is similar to the IEC-104 module found in [Industroyer](https://attack.mitre.org/software/S0604). Security researchers assess that [Industroyer2](https://attack.mitre.org/software/S1072) was designed to cause impact to high-voltage electrical substations. The initial [Industroyer2](https://attack.mitre.org/software/S1072) sample was compiled on 03/23/2022 and scheduled to execute on 04/08/2022, however it was discovered before deploying, resulting in no impact.(Citation: Industroyer2 Blackhat ESET)'}, {'malware.name': 'QUADAGENT', 'malware.description': '[Q


> Finished chain.
Industroyer2, QUADAGENT, PLAINTEE, Avenger, CARROTBAT,
BoxCaon, Cardinal RAT, OnionDuke, build_downer, Pteranodon,
DealersChoice, PS1, Spark, Olympic Destroyer, CostaBricks,
SYSCON, PowerPunch, POWERSTATS, Cannon, BadPatch,
Final1stspy, RogueRobin, DDKONG, PyDCrypt, DCSrv,
Industroyer can attack DCConverterUnit.


'Industroyer2, QUADAGENT, PLAINTEE, Avenger, CARROTBAT, BoxCaon, Cardinal RAT, OnionDuke, build_downer, Pteranodon, DealersChoice, PS1, Spark, Olympic Destroyer, CostaBricks, SYSCON, PowerPunch, POWERSTATS, Cannon, BadPatch, Final1stspy, RogueRobin, DDKONG, PyDCrypt, DCSrv, Industroyer can attack DCConverterUnit.'

In [43]:
a = prettyCypherChain("Give Malwares that can attack DCConverterUnit?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (malware:Malware)-[:possible_attack]->(node:Interface)
    WHERE node.name = 'DCConverterUnit'
RETURN malware.name, malware.description
Full Context:
[{'malware.name': 'Industroyer2', 'malware.description': '[Industroyer2](https://attack.mitre.org/software/S1072) is a compiled and static piece of malware that has the ability to communicate over the IEC-104 protocol. It is similar to the IEC-104 module found in [Industroyer](https://attack.mitre.org/software/S0604). Security researchers assess that [Industroyer2](https://attack.mitre.org/software/S1072) was designed to cause impact to high-voltage electrical substations. The initial [Industroyer2](https://attack.mitre.org/software/S1072) sample was compiled on 03/23/2022 and scheduled to execute on 04/08/2022, however it was discovered before deploying, resulting in no impact.(Citation: Industroyer2 Blackhat ESET)'}, {'malware.name': 'QUADAGENT', 'malware.description':


> Finished chain.
Industroyer, Industroyer2, and DCSrv are malwares that can
attack DCConverterUnit.


In [44]:
prettyCypherChain("How to defend against malwares Industroyer?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (malware:Malware {name: 'Industroyer'})-[:revoked_by]->(defense)
RETURN defense.name, defense.description
Full Context:
[]

> Finished chain.
I don't know the answer.


"I don't know the answer."

# LLM based Link connection

In [45]:
cypher = """

    MATCH (n:Interface)
    RETURN n.id as id, n.name AS name, n.comment AS comment, n.description as description

"""

powergrid_nodes = kg.query(cypher)

In [46]:
powergrid_nodes

[{'id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:EquipmentContainer;1',
  'name': None,
  'comment': None,
  'description': None},
 {'id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:ConnectivityNodeContainer;1',
  'name': 'ConnectivityNodeContainer',
  'comment': 'Adapted from CIM and https://github.com/smart-data-models/dataModel.EnergyCIM/ data models',
  'description': 'A base class for all objects that may contain connectivity nodes or topological nodes.'},
 {'id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:Equipment;1',
  'name': 'relatedEquipment',
  'comment': 'connection to an equipment.',
  'description': None},
 {'id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:BaseVoltage;1',
  'name': 'BaseVoltage',
  'comment': 'Base voltage of the transformer end.  This is essential for PU calculation.',
  'description': None},
 {'id': 'dtmi:digitaltwins:ngsi_ld:cim:energy:IdentifiedObject;1',
  'name': 'IdentifiedObject',
  'comment': 'Adapted from CIM and https://github.com/smart-data-models/dataModel.En

In [47]:



# for each node
# remove existig links formed through cosine similarity analysis
# get RAG model output for each node
# parse the output and then form the links

In [53]:
def get_rag_prediction(question):
    return prettyCypherChain(question)

def parse_rag_output(response):
    ret_list = []
    if(response == "I don't know the answer."):
        return None
    
    
    #for multiple values
    malwares = [x.strip() for x in response.split(',')]
    
    ret_list.extend(malwares[:-1])
    
    #for the last value
    last = malwares[-1].strip().split(" ")
    
    if(last[0] == 'and'):
        ret_list.append(last[1])
    else:
        ret_list.append(last[0])
    return ret_list

def delete_existing_links(kg, node):
    query = "MATCH ()-[r:possible_attack]->(i:Interface {id: $node_id}) DELETE r"
    params = {
        'node_id' : node.get('id')
    }
     # Construct the final query by replacing parameters
    final_query = query.replace("$node_id", f"'{params['node_id']}'")
    
    # Print the final formed query
    print("Final Delete Query:", final_query)
    
    status = kg.query(query, params)
    print(status)
    return status

def create_links(kg, node, malware):
    
    
    print(node.get('name'))
    print(malware)

    query = "\
    MATCH (i:Interface {id: $node_id}), (m:Malware {name: $malware})\
    MERGE (m)-[:possible_attack]->(i)\
    "
    params = {
        "node_id":node.get('id'), 
        "malware":malware
    }
    kg.query(query,params)
    status = kg.query(query, params)
    print("Create status:", status)
    return status
    
    

def create_new_links_for_node(kg, node):
    node_id = node.get("id")
    node_name = node.get("name")
    node_comment = node.get("comment")
    node_desc = node.get("description")
    details = ""
    if(None != node_comment and not node_comment.startswith("Adapted from CIM")):
        details += node_comment
    elif (None != node_desc and not node_desc.startswith("Adapted from CIM")):
        details += node_desc
        
    question = f"""Give Malwares that can attack {node_name}?"""
    
    print(question)
    response = get_rag_prediction(question)
    malwares = parse_rag_output(response)
    print(malwares)
    
    # delete existing links formed form similarity score
    delete_existing_links(kg, node)
    
    # form new links if exists
    if(malwares):
        for malware in malwares:
            status = create_links(kg, node, malware)
        return status
    return None 



In [54]:
for node in powergrid_nodes:
    create_new_links_for_node(kg, node)

Give Malwares that can attack None?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (malware:Malware)-[:possible_attack]->(node:Interface)
    WHERE node.name = 'None'
RETURN malware.name, malware.description
Full Context:
[]

> Finished chain.
I don't know the answer.
None
Final Delete Query: MATCH ()-[r:possible_attack]->(i:Interface {id: 'dtmi:digitaltwins:ngsi_ld:cim:energy:EquipmentContainer;1'}) DELETE r
[]
Give Malwares that can attack ConnectivityNodeContainer?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (malware:Malware)-[:possible_attack]->(node:Interface)
    WHERE node.name = 'ConnectivityNodeContainer'
RETURN malware.name, malware.description
Full Context:
[{'malware.name': 'Siloscape', 'malware.description': '[Siloscape](https://attack.mitre.org/software/S0623) is malware that targets Kubernetes clusters through Windows containers. [Siloscape](https://attack.mitre.org/software/S0623) was first observed in March 2021.(Citation:


> Finished chain.
YAHOYAH, Hydraq, Hacking Team UEFI Rootkit, CozyCar, Rising
Sun, ThiefQuest, UBoatRAT, Prikormka, Cobalt Strike,
SHIPSHAPE, COATHANGER, MobileOrder, KARAE, EvilGrab, HDoor,
BlackEnergy, Industroyer, Industroyer2, SPACESHIP.
['YAHOYAH', 'Hydraq', 'Hacking Team UEFI Rootkit', 'CozyCar', 'Rising Sun', 'ThiefQuest', 'UBoatRAT', 'Prikormka', 'Cobalt Strike', 'SHIPSHAPE', 'COATHANGER', 'MobileOrder', 'KARAE', 'EvilGrab', 'HDoor', 'BlackEnergy', 'Industroyer', 'Industroyer2', 'SPACESHIP.']
Final Delete Query: MATCH ()-[r:possible_attack]->(i:Interface {id: 'dtmi:digitaltwins:ngsi_ld:cim:energy:Equipment;1'}) DELETE r
[]
relatedEquipment
YAHOYAH
Create status: []
relatedEquipment
Hydraq
Create status: []
relatedEquipment
Hacking Team UEFI Rootkit
Create status: []
relatedEquipment
CozyCar
Create status: []
relatedEquipment
Rising Sun
Create status: []
relatedEquipment
ThiefQuest
Create status: []
relatedEquipment
UBoatRAT
Create status: []
relatedEquipment
Prikormka
Create s


> Finished chain.
I don't know the answer.
None
Final Delete Query: MATCH ()-[r:possible_attack]->(i:Interface {id: 'dtmi:digitaltwins:ngsi_ld:cim:energy:BaseVoltage;1'}) DELETE r
[]
Give Malwares that can attack IdentifiedObject?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (malware:Malware)-[:possible_attack]->(node:Interface)
    WHERE node.name = 'IdentifiedObject'
RETURN malware.name, malware.description
Full Context:
[{'malware.name': 'Olympic Destroyer', 'malware.description': '[Olympic Destroyer](https://attack.mitre.org/software/S0365) is malware that was used by [Sandworm Team](https://attack.mitre.org/groups/G0034) against the 2018 Winter Olympics, held in Pyeongchang, South Korea. The main purpose of the malware was to render infected computer systems inoperable. The malware leverages various native Windows utilities and API calls to carry out its destructive tasks. [Olympic Destroyer](https://attack.mitre.org/software/S0365) has worm-like features 


> Finished chain.
Regin, Snip3, Gold Dragon, BlackEnergy, Rover, TrailBlazer,
OceanSalt, NavRAT, Industroyer, Valak, ECCENTRICBANDWAGON,
WarzoneRAT, Prestige, UBoatRAT, BADNEWS, Attor, HALFBAKED,
HARDRAIN, NativeZone, WannaCry, LOWBALL, SLOWDRIFT,
ADVSTORESHELL, OnionDuke, SOUNDBITE, RobbinHood, ROKRAT,
S-Type, RunningRAT, MoonWind, RATANKBA, StoneDrill, Cuba,
Bankshot, Olympic Destroyer, Ixeshe, CostaBricks, Action
RAT, Hi-Zor, Astaroth, REvil, YAHOYAH, Bad Rabbit, Aria-
body, KARAE, Prikormka, PS1, AcidRain.
['Regin', 'Snip3', 'Gold Dragon', 'BlackEnergy', 'Rover', 'TrailBlazer', 'OceanSalt', 'NavRAT', 'Industroyer', 'Valak', 'ECCENTRICBANDWAGON', 'WarzoneRAT', 'Prestige', 'UBoatRAT', 'BADNEWS', 'Attor', 'HALFBAKED', 'HARDRAIN', 'NativeZone', 'WannaCry', 'LOWBALL', 'SLOWDRIFT', 'ADVSTORESHELL', 'OnionDuke', 'SOUNDBITE', 'RobbinHood', 'ROKRAT', 'S-Type', 'RunningRAT', 'MoonWind', 'RATANKBA', 'StoneDrill', 'Cuba', 'Bankshot', 'Olympic Destroyer', 'Ixeshe', 'CostaBricks', 'Action RAT',


> Finished chain.
Bankshot, Industroyer, Cyclops Blink, Chaos, Bisonal,
ShadowPad, OceanSalt, Regin, BlackEnergy, Olympic Destroyer,
LoJax, TinyTurla, CostaBricks, PlugX, RobbinHood
['Bankshot', 'Industroyer', 'Cyclops Blink', 'Chaos', 'Bisonal', 'ShadowPad', 'OceanSalt', 'Regin', 'BlackEnergy', 'Olympic Destroyer', 'LoJax', 'TinyTurla', 'CostaBricks', 'PlugX', 'RobbinHood']
Final Delete Query: MATCH ()-[r:possible_attack]->(i:Interface {id: 'dtmi:digitaltwins:ngsi_ld:cim:energy:Location;1'}) DELETE r
[]
Location
Bankshot
Create status: []
Location
Industroyer
Create status: []
Location
Cyclops Blink
Create status: []
Location
Chaos
Create status: []
Location
Bisonal
Create status: []
Location
ShadowPad
Create status: []
Location
OceanSalt
Create status: []
Location
Regin
Create status: []
Location
BlackEnergy
Create status: []
Location
Olympic Destroyer
Create status: []
Location
LoJax
Create status: []
Location
TinyTurla
Create status: []
Location
CostaBricks
Create status: []
Locat

...


> Finished chain.
I don't know the answer.
None
Final Delete Query: MATCH ()-[r:possible_attack]->(i:Interface {id: 'dtmi:digitaltwins:ngsi_ld:cim:energy:FossilFuel;1'}) DELETE r
[]
Give Malwares that can attack NuclearGeneratingUnit?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (malware:Malware)-[:possible_attack]->(node:Interface)
    WHERE node.name = 'NuclearGeneratingUnit'
RETURN malware.name, malware.description
Full Context:
[{'malware.name': 'Industroyer', 'malware.description': '[Industroyer](https://attack.mitre.org/software/S0604) is a sophisticated malware framework designed to cause an impact to the working processes of Industrial Control Systems (ICS), specifically components used in electrical substations.(Citation: ESET Industroyer) [Industroyer](https://attack.mitre.org/software/S0604) was used in the attacks on the Ukrainian power grid in December 2016.(Citation: Dragos Crashoverride 2017) This is the first publicly known malware specifically 


> Finished chain.
I don't know the answer.
None
Final Delete Query: MATCH ()-[r:possible_attack]->(i:Interface {id: 'dtmi:digitaltwins:ngsi_ld:cim:energy:NuclearGeneratingUnit;1'}) DELETE r
[]
Give Malwares that can attack PowerElectronicsWindUnit?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (malware:Malware)-[:possible_attack]->(node:Interface)
    WHERE node.name = 'PowerElectronicsWindUnit'
RETURN malware.name, malware.description
Full Context:
[{'malware.name': 'POWERSTATS', 'malware.description': '[POWERSTATS](https://attack.mitre.org/software/S0223) is a PowerShell-based first stage backdoor used by [MuddyWater](https://attack.mitre.org/groups/G0069). (Citation: Unit 42 MuddyWater Nov 2017)'}, {'malware.name': 'NETEAGLE', 'malware.description': '[NETEAGLE](https://attack.mitre.org/software/S0034) is a backdoor developed by [APT30](https://attack.mitre.org/groups/G0013) with compile dates as early as 2008. It has two main variants known as “Scout” and “No


> Finished chain.
I don't know the answer.
None
Final Delete Query: MATCH ()-[r:possible_attack]->(i:Interface {id: 'dtmi:digitaltwins:ngsi_ld:cim:energy:PowerElectronicsWindUnit;1'}) DELETE r
[]
Give Malwares that can attack BatteryUnit?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (malware:Malware)-[:possible_attack]->(node:Interface)
    WHERE node.name = 'BatteryUnit'
RETURN malware.name, malware.description
Full Context:
[{'malware.name': 'BlackEnergy', 'malware.description': '[BlackEnergy](https://attack.mitre.org/software/S0089) is a malware toolkit that has been used by both criminal and APT actors. It dates back to at least 2007 and was originally designed to create botnets for use in conducting Distributed Denial of Service (DDoS) attacks, but its use has evolved to support various plug-ins. It is well known for being used during the confrontation between Georgia and Russia in 2008, as well as in targeting Ukrainian institutions. Variants include Blac


> Finished chain.
I don't know the answer.
None
Final Delete Query: MATCH ()-[r:possible_attack]->(i:Interface {id: 'dtmi:digitaltwins:ngsi_ld:cim:energy:BatteryUnit;1'}) DELETE r
[]
Give Malwares that can attack fossilFuelType?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (malware:Malware)-[:possible_attack]->(node:Interface)
    WHERE node.name = 'fossilFuelType'
RETURN malware.name, malware.description
Full Context:
[{'malware.name': 'Flame', 'malware.description': '[Flame](https://attack.mitre.org/software/S0143) is a sophisticated toolkit that has been used to collect information since at least 2010, largely targeting Middle East countries. (Citation: Kaspersky Flame)'}, {'malware.name': 'AutoIt backdoor', 'malware.description': '[AutoIt backdoor](https://attack.mitre.org/software/S0129) is malware that has been used by the actors responsible for the MONSOON campaign. The actors frequently used it in weaponized .pps files exploiting CVE-2014-6352. (Citatio


> Finished chain.
I don't know the answer.
None
Final Delete Query: MATCH ()-[r:possible_attack]->(i:Interface {id: 'dtmi:digitaltwins:ngsi_ld:cim:energy:FuelType;1'}) DELETE r
[]


In [270]:
#question = "Give Malwares that can attack DCConverterUnit (The DC converter unit belonging of the substation.)?"

In [271]:
#response = get_rag_prediction(question)
#malwares = parse_rag_output(response)
#malwares

In [246]:
b = a.split(',')
b

['Industroyer', ' Industroyer2 can attack DCConverterUnit.']

In [244]:
b[:-1]

['Industroyer']

In [251]:
b[-1].strip().split(" ")

['Industroyer2', 'can', 'attack', 'DCConverterUnit.']

# Delete possible attack relation

In [417]:
def delete_all_possible_attack_relations(kg):
    cypher = """
    MATCH ()-[r:possible_attack]->()
    DELETE r
    RETURN count(r) as deleted_count
    """
    result = kg.query(cypher)
    deleted_count = result[0]["deleted_count"] if result else 0
    print(f"Deleted {deleted_count} possible_attack relationships.")
    return deleted_count

In [468]:
delete_all_possible_attack_relations(kg)

Deleted 8152 possible_attack relationships.


8152